# Text pipeline
Text Pipeline: 

Current sentence + Emotion + context + NER + keyword  --[T5-fintuned-CommonGen]--> next sentence

In [1]:
!pip install SentencePiece
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install bert_score
!pip install sacrebleu

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JAM_0\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JAM_0\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JAM_0\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Stry233/A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability.git

Cloning into 'A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability'...
remote: Enumerating objects: 890, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 890 (delta 81), reused 169 (delta 81), pack-reused 718
Receiving objects: 100% (890/890), 202.97 MiB | 27.14 MiB/s, done.
Resolving deltas: 100% (572/572), done.
Checking out files: 100% (299/299), done.


## Current sentence

In [1]:
import pandas as pd
storyData = pd.read_csv('../../data/csv_version/dev/emotion/allcharlinepairs-clean-v1.csv') # local
# storyData = pd.read_csv('./A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability/data/csv_version/dev/emotion/allcharlinepairs-clean-v1.csv', index_col=0)
def getAStory(storyData):
    sentenceIndex = storyData[storyData['context'].isna()].sample()['sentence'].index
    demoStory = storyData.iloc[sentenceIndex[0]:sentenceIndex[0]+5]
    return demoStory

demoStory = getAStory(storyData)
demoSentence = demoStory.iloc[0]['sentence']
print(demoSentence)
demoStory.style.set_properties(subset=['context'], **{'width-min': '300px'})

I had always wanted to go to south america.


,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
4782,4782,90ff0ba5-4481-4fec-87fb-97dbab9fd294,1,I (myself),ann0,nan,I had always wanted to go to south america.,yes,"['desire', 'ambitious']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 0, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
4783,4783,90ff0ba5-4481-4fec-87fb-97dbab9fd294,2,I (myself),ann0,I had always wanted to go to south america.,I saved up as much money as I could for a year.,yes,"['excited', 'proud', 'frugal']","{'joy': 3, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
4784,4784,90ff0ba5-4481-4fec-87fb-97dbab9fd294,3,I (myself),ann0,I had always wanted to go to south america.|I saved up as much money as I could for a year.,I took my vacation from work after saving enough money.,yes,"['glad', 'accomplished', 'satisfied', 'happy']","{'joy': 3, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
4785,4785,90ff0ba5-4481-4fec-87fb-97dbab9fd294,4,I (myself),ann0,I had always wanted to go to south america.|I saved up as much money as I could for a year.|I took my vacation from work after saving enough money.,I travelled to a very nice hotel.,yes,"['enjoyment', 'satisfied', 'happy']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
4786,4786,90ff0ba5-4481-4fec-87fb-97dbab9fd294,5,I (myself),ann0,I had always wanted to go to south america.|I saved up as much money as I could for a year.|I took my vacation from work after saving enough money.|I travelled to a very nice hotel.,I went to exotic sites and swam at the beach.,yes,"['fulfilled', 'excited', 'happy', 'blissful']","{'joy': 3, 'trust': 0, 'fear': 0, 'surprise': 0, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 1}"


In [2]:
storyData = pd.read_csv('../../data/csv_version/dev/emotion/allcharlinepairs-clean-v1.csv') # local
def getAStory(storyData, index):
    sentenceIndex = storyData.iloc[index]
    demoStory = storyData[sentenceIndex[0]:sentenceIndex[0]+5]
    return demoStory

demoStory = getAStory(storyData, 876)
demoSentence = demoStory.iloc[0]['sentence']
print(demoSentence)
demoStory.style.set_properties(subset=['context'], **{'width-min': '300px'})

I wanted to do a different thing than the usual.


,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
876,876,3b2a458e-3967-4eb6-acf9-1b56c6fc23b0,1,I (myself),ann0,nan,I wanted to do a different thing than the usual.,yes,"['optimistic', 'excited', 'spontaneous', 'adventurous']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 3}"
877,877,3b2a458e-3967-4eb6-acf9-1b56c6fc23b0,2,I (myself),ann0,I wanted to do a different thing than the usual.,I got into my car and went on the road.,yes,"['surprised', 'joy', 'positive']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 0}"
878,878,3b2a458e-3967-4eb6-acf9-1b56c6fc23b0,3,I (myself),ann0,I wanted to do a different thing than the usual.|I got into my car and went on the road.,I stopped at a blockbuster video store.,yes,"['spontaneous', 'excited', 'adventurous']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
879,879,3b2a458e-3967-4eb6-acf9-1b56c6fc23b0,4,I (myself),ann0,I wanted to do a different thing than the usual.|I got into my car and went on the road.|I stopped at a blockbuster video store.,I went inside and rented a movie.,yes,"['bored', 'happy', 'positive']","{'joy': 2, 'trust': 0, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}"
880,880,3b2a458e-3967-4eb6-acf9-1b56c6fc23b0,5,I (myself),ann0,I wanted to do a different thing than the usual.|I got into my car and went on the road.|I stopped at a blockbuster video store.|I went inside and rented a movie.,I brought the movie home and watched the whole thing.,yes,"['satisfied', 'happiness', 'joy']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 1}"


## Emotion

In [3]:
import warnings
import pandas as pd
import ast
import random

In [4]:
attributes = ['joy','trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation']
# dataDevPlutchik = pd.read_csv('allcharlinepairs.csv') #text file
dataDevPlutchik = storyData
dataDevPlutchik['plutchik'] = [ast.literal_eval(x) for x in dataDevPlutchik['plutchik']]
print(dataDevPlutchik['plutchik'][0], len(dataDevPlutchik['plutchik'][0]))

{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': 1, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2} 8


In [5]:
emoratio = {'joy': 0, 'trust': 0, 'fear': 0, 'surprise': 0, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 0}
emoratio

{'joy': 0,
 'trust': 0,
 'fear': 0,
 'surprise': 0,
 'sadness': 0,
 'disgust': 0,
 'anger': 0,
 'anticipation': 0}

In [6]:
results = dict()

currid = dataDevPlutchik['storyid'][0]
counter = 0
labels = []
emotions = []
ans = []
err = 0
trainset = []
while(counter < len(dataDevPlutchik)):
    tcounter = 0
    while(dataDevPlutchik['storyid'][counter + tcounter]==currid):
        tstr = ""
        for k in dataDevPlutchik['plutchik'][counter+tcounter].keys():
            if(dataDevPlutchik['plutchik'][counter+tcounter][k] > 0):
                tstr += str(1)
                emoratio[k] += 1
            else:
                tstr += str(0)
            
        
        tcounter += 1
        if(counter + tcounter >= len(dataDevPlutchik)):
            break
        
        if(tstr != ''):
            if(tstr not in results.keys()):
                results[tstr] = [0 for u in range(9)]
            results[tstr][8] += 1
            if(dataDevPlutchik['storyid'][counter+tcounter+1] == currid):
                tstr1 = ""
                for k in dataDevPlutchik['plutchik'][counter+tcounter+1].keys():
                    if(int(dataDevPlutchik['plutchik'][counter+tcounter+1][k]) > 0):
                        tstr1 += '1'
                        num = attributes.index(k)
                        results[tstr][num] += 1
                    else:
                        tstr1 += '0'
                trainset.append(tstr+' '+tstr1)       
    currid = dataDevPlutchik['storyid'][counter+tcounter]
    counter += tcounter
    if(counter + tcounter >= len(dataDevPlutchik)):
        break 
    
print(len(results))

print(results)

241
{'11010001': [487, 372, 169, 461, 180, 127, 129, 506, 1462], '11000001': [717, 525, 308, 659, 333, 236, 255, 766, 1563], '00110110': [6, 3, 6, 11, 4, 8, 5, 6, 33], '00011110': [24, 18, 23, 59, 49, 36, 42, 39, 196], '00001100': [14, 9, 5, 14, 6, 8, 5, 14, 35], '10001001': [15, 11, 6, 10, 10, 4, 7, 18, 35], '10010001': [142, 89, 56, 135, 54, 47, 45, 136, 416], '11000000': [101, 85, 71, 113, 83, 50, 54, 122, 356], '10000000': [122, 86, 61, 125, 73, 57, 62, 132, 369], '00000110': [9, 5, 9, 15, 18, 19, 24, 12, 57], '00000010': [8, 5, 5, 9, 5, 5, 8, 9, 34], '11111111': [26, 21, 16, 36, 22, 15, 15, 27, 70], '10000010': [2, 0, 1, 3, 1, 3, 4, 1, 11], '10010000': [33, 20, 19, 40, 21, 16, 13, 35, 157], '10111101': [3, 2, 1, 1, 2, 1, 1, 3, 11], '00101011': [10, 11, 6, 9, 6, 2, 3, 13, 24], '00101111': [48, 27, 34, 46, 34, 29, 27, 54, 108], '00101101': [10, 9, 9, 9, 9, 5, 6, 13, 33], '10111001': [8, 7, 5, 7, 3, 2, 1, 9, 17], '11100001': [55, 40, 43, 58, 34, 28, 32, 75, 136], '10000001': [318, 21

In [ ]:
emoratio

{'anger': 3065,
 'anticipation': 7955,
 'disgust': 2856,
 'fear': 3910,
 'joy': 6722,
 'sadness': 3842,
 'surprise': 6139,
 'trust': 4816}

In [ ]:
emorate = [round(emoratio[key]/1000) for key in emoratio.keys()]
emorate

[7, 5, 4, 6, 4, 3, 3, 8]

# GET NEXT SENTENCE EMOTION(Probability Distribution + Distance)

In [ ]:
for k in results.keys():
    for u in range(len(results[k])-1):
        results[k][u] = results[k][u]/results[k][8]/emorate[u]
results

In [ ]:
def getEmoDist(emolastsent):
    ret = sorted(set(random.choices(attributes, results[emolastsent][:8], k=8)))
    if(emolastsent in results.keys()):
        return ', '.join(list(ret))
    emokey = list(results.keys())
    mindist = 1000
    ridx = ""
    for i in range(len(emokey)):
        dist = 0
        for j in range(len(emokey[i])):
            dist += ((int(emokey[i][j])-int(emolastsent[j]))**2)**(0.5)
        if(dist<mindist):
            mindist = dist
            ridx = emokey[i]
    ret = sorted(set(random.choices(attributes, results[ridx][:8], k=8)))
    return ', '.join(list(ret))

# GET NEXT SENTENCE EMOTION(Bert Model)

In [7]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np

# joy, trust, fear, surprise, sadness, disgust, anger, anticipation
emotionActivate =  {'joy' : True, 
            'trust' : True, 
            'fear' : True, 
            'surprise' : True, 
            'sadness' : True, 
            'disgust' : True, 
            'anger' : True, 
            'anticipation' : True}

import torch
def genEmo(demoSentenceInput):
    # model = AutoModelForSequenceClassification.from_pretrained("./firstSentenceDetection/") # local
    model = AutoModelForSequenceClassification.from_pretrained("Yuetian/bert-base-uncased-finetuned-plutchik-emotion") # local
    # assert model.config.output_attentions == True
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
      
    encoding = tokenizer(demoSentenceInput, return_tensors="pt")
    # tokenizer.decode(encoding['input_ids'])
    encoding = {k: v.to('cpu') for k,v in encoding.items()}

    outputs = model(**encoding)
    
    # apply sigmoid + threshold
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.5)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [list(emotionActivate.keys())[idx] for idx, label in enumerate(predictions) if label == 1.0 and list(emotionActivate.values())[idx] == True]
    return ", ".join( key for key in predicted_labels)

In [8]:
genEmo("I am happy")

'joy, trust, anticipation'

# NER

In [9]:
!pip install SceneGraphParser
!python -m spacy download en  
!pip install spacy

     ---------------------------------------- 12.8/12.8 MB 5.0 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import sng_parser
# TO-DO: GPT 2 generate keyword
def genKeyword(demoSentenceInput):
    graph = sng_parser.parse(demoSentenceInput)
    # print(graph)
    majorKeyword = [x['span'] for x in graph['entities']]
    return ", ".join( key for key in majorKeyword)

In [11]:
demoSentenceInput = "I and Ann were having a night walk yesterday"
graph = sng_parser.parse(demoSentenceInput)
graph

{'entities': [{'span': 'I',
   'lemma_span': 'I',
   'head': 'I',
   'lemma_head': 'I',
   'span_bounds': (0, 1),
   'modifiers': [],
   'type': 'unknown'},
  {'span': 'Ann',
   'lemma_span': 'Ann',
   'head': 'Ann',
   'lemma_head': 'Ann',
   'span_bounds': (2, 3),
   'modifiers': [],
   'type': 'unknown'},
  {'span': 'a night walk',
   'lemma_span': 'a night walk',
   'head': 'night walk',
   'lemma_head': 'night walk',
   'span_bounds': (5, 8),
   'modifiers': [{'dep': 'det', 'span': 'a', 'lemma_span': 'a'}],
   'type': 'unknown'}],
 'relations': [{'subject': 0,
   'object': 2,
   'relation': 'having',
   'lemma_relation': 'have'},
  {'subject': 1, 'object': 2, 'relation': 'having', 'lemma_relation': 'have'}]}

In [8]:
genKeyword("I brought the movie home and watched the whole thing.")

'I, the movie, the whole thing'

# Image Generation

## PreProcess text prompt


In [ ]:
#Carl Spitzweg (decent
#Miyazaki Hayao (not working so well
#Cyberpunk (ok....)
#cartoon (not that ok..)
#Trending on artstation.

myPrompt = {"0": ["One person gathered his friends to tell them a funny joke he heard.","Carl Spitzweg ","Trending on artstation.", "country view"]} #@param type="raw"

## Set Up disco diffusion Environment

In [ ]:
#@title Set Up Environment
import os, sys
import subprocess, torch

#@markdown Leave these as defaults unless you know what you are doing.

use_google_drive = True #@param {type:"boolean"}
save_models = True #@param {type:"boolean"}
check_for_updates = True #@param {type:"boolean"}
content_root = '/content'
repo = "https://github.com/entmike/disco-diffusion-1" #@param {type:"string"}
branch = "main" #@param {type:"string"}
cwd = os.path.abspath('.')
is_local=True

if is_local: content_root=cwd
print (f'Current directory: {cwd}')

if use_google_drive == True:
  import os
  from google.colab import drive
  if os.path.isdir('/content/gdrive') == False:
    print(f'📁 Mounting Google Drive.  Please accept any confirmation screens.')
    drive.mount('/content/gdrive/')
  else:
    print(f'📁 Google Drive already mounted.')
  content_root = '/content/gdrive/MyDrive'

dd_root = f'{content_root}/disco-diffusion-1'

print(f'✅ Disco Diffusion root path will be "{dd_root}"')

is_colab = False
try:
    from google.colab import drive
    print("Google Colab detected.")
    is_colab=True
except:
    print("Google Colab not detected.")
    is_colab=False

root_path = dd_root

# Clone Repo
if os.path.isdir(f'{dd_root}') == False:
  print(f"Cloning repo '{repo}' into '{dd_root}'...")
  os.chdir(f'{content_root}')
  subprocess.run(f'git clone {repo}'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")

os.chdir(f'{dd_root}')
if check_for_updates == True:
  # Pull any updates
  print(f'📄 Pulling updates from GitHub...')
  for cmd in ['git clean -df', f'git checkout {branch}', f'git pull origin {branch}']:
    gitresults = subprocess.run(f'{cmd}'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")
    print(f'{gitresults}')
else:
  print("⚠️ Skipping checking for Git updates")

# Downgrade for T4/V100
device_name = torch.cuda.get_device_name(0)
print(f"🔍 You have a {device_name} GPU.")

#Upgrade pyyaml if in Colab
if is_colab:
    print(f'📦 Upgrading pyyaml...')
    subprocess.run(f'pip install --upgrade pyyaml --quiet'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")
    
    if "V100" in device_name or "T4" in device_name:
      print(f"⚠️ {torch.cuda.get_device_name(0)} detected.  Do not use ViTL14 or ViTL14@336 models...")
      # This causes problems with pytorch3d sigh... -Mike (5/13/22)
      # print("📦 Downgrading pytorch...")
      # downgrade_torch = subprocess.run('pip install torch==1.10.2 torchvision==0.11.3 -q'.split(' '), stdout=subprocess.PIPE).stdout.decode('utf-8')
      # import torch
    
    print(f'📦 Installing pip requirements...')
    piplog = subprocess.run(f'pip install -r colab-requirements.txt'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")
    # print(piplog)
# Set base project directory to current working directory
PROJECT_DIR = dd_root

# Import DD helper modules
sys.path.append(PROJECT_DIR)
import dd, dd_args

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


## Set up Disco Diffusion Parameters

In [ ]:
from pydotted import pydot
args = pydot({})

#@markdown ##💡Batch/Config Parameters
#@markdown Specifies the folder name and filenames created in `images_out` to store your images.
args.batch_name = "TimeToDisco" #@param {type:"string"}
#@markdown If your batch run gets interrupted (either because you stopped it, or because of a disconnection) you can resume your batch run where you left off using this setting.  However, you MUST not change the settings in the batch, or it cannot reliably be resumed.  Other parameters (`run_to_resume`, `resume_from_frame`, `retain_overwritten_frames`) control how you want the batch to be resumed.  If you have interrupted a run and tweaked settings, you should NOT use `resume_run`, as this would be considered a new run with the new settings.
args.resume_run = False #@param {type:"boolean"}
args.run_to_resume = "latest" #@param {type:"string"}
args.retain_overwritten_frames = False #@param {type:"boolean"}
#@markdown ## 🔍Preview Settings
args.intermediate_saves = 0 #@param {type:"number"}
args.intermediates_in_subfolder = True #@param {type:"boolean"}
args.console_preview = False #@param {type:"boolean"}
args.console_preview_width = 80 #@param {type:"number"}
args.display_rate = 50 #@param {type:"number"}

args.config_file = None #@param {type: "raw"}
#@markdown Accepts any hard-coded integer as a random seed.  A value of `'random_seed'` will randomly generate a seed at runtime.
args.set_seed = 'random_seed' #@param type="raw"
#@markdown How many batches to run.  (Ignored with animation modes)
args.n_batches =  3#@param {type:"number"}
#@markdown Resolution of output image.  Must be a multiple of 64.
args.width_height = [1280,768] #@param {type:"raw"}
#@markdown How many iterations per image are taken
args.steps = 250 #@param [25,50,100,150,250,500,1000]{type: 'raw', allow-input: true}
# 📝 Prompts Settings
args.text_prompts = myPrompt

args.prompt_salad = False 

args.prompt_salad_template = "{colors} {things} in the {custom/customword} shape of {shapes}, art by {artists}" #@param

args.prompt_salad_amount = 5 
args.fuzzy_prompt = False 
#@markdown ## 🖼️ Init Image Settings
#@markdown (Optional) Starting image you wish to use
args.init_image = None #@param {type:"raw"}
args.init_scale = 1000 #@param {type:"number"}
#@markdown Make sure you set skip_steps to ~50% of your steps if you want to use an init image.
args.skip_steps =  0#@param {type:"number"}
#@markdown ## 💥 Perlin/Voronoi Settings
#@markdown Select between Perlin noise or Voronoi diagram as a starting point
#@markdown Make sure you set skip_steps to ~25-50% of your steps if you want to use the voronoi diagram.
args.init_generator = "perlin" # @param ["perlin","voronoi"]
args.perlin_init = False #@param {type:"boolean"}
args.perlin_mode = "mixed" #@param ["mixed", "color", "gray"]
args.voronoi_points = 20 #@param [20,50,100,150,250,500,1000]{type: 'raw', allow-input: true}
args.voronoi_palette = 'default.yaml' #@param{type:'string'}
#@markdown ---
#@markdown ## 🧠 Models
#@markdown These various CLIP models are available for you to use during image generation. Models have different styles or "flavors", so look around.
#@markdown
#@markdown `use_secondary_model` is to use a secondary purpose-made diffusion model to clean up interim diffusion images for CLIP evaluation. If this option is turned off, DD will use the regular (large) diffusion model. Using the secondary model is faster - one user reported a 50% improvement in render speed. However, the secondary model is much smaller, and may reduce image quality and detail. It is suggested that you experiment with this.
#@markdown You can mix in multiple models as well for different results. However, keep in mind that some models are extremely memory-hungry, and turning on additional models will take additional memory and may cause a crash.
#@markdown
#@markdown Default Values and the rough order of speed/mem usage is (smallest/fastest to largest/slowest):
args.RN50 = True #@param {type:"boolean"}
args.RN101 = False #@param {type:"boolean"}
args.RN50x64 = False #@param {type:"boolean"}
args.RN50x16 = False #@param {type:"boolean"}
args.RN50x4 = False #@param {type:"boolean"}
args.ViTB16 = True #@param {type:"boolean"}
args.ViTB32 = True #@param {type:"boolean"}
args.ViTL14 = False #@param {type:"boolean"}
args.ViTL14_336 = False #@param {type:"boolean"}
#@markdown Open CLIP Models
args.ViTB32_laion2b_e16 = False #@param {type:"boolean"}
args.ViTB32_laion400m_e31 = False #@param {type:"boolean"}
args.ViTB32_laion400m_e32 = False #@param {type:"boolean"}
args.ViTB32quickgelu_laion400m_e31 = False #@param {type:"boolean"}
args.ViTB32quickgelu_laion400m_e32 = False #@param {type:"boolean"}
args.ViTB16_laion400m_e31 = False #@param {type:"boolean"}
args.ViTB16_laion400m_e32 = False #@param {type:"boolean"}
args.RN50_yffcc15m = False #@param {type:"boolean"}
args.RN50_cc12m = False #@param {type:"boolean"}
args.RN50_quickgelu_yfcc15m = False #@param {type:"boolean"}
args.RN50_quickgelu_cc12m = False #@param {type:"boolean"}
args.RN101_yfcc15m = False #@param {type:"boolean"}
args.RN101_quickgelu_yfcc15m = False #@param {type:"boolean"}
args.use_secondary_model = True #@param {type:"boolean"}
args.diffusion_model = "512x512_diffusion_uncond_finetune_008100" #@param ["512x512_diffusion_uncond_finetune_008100","256x256_diffusion_uncond","pixel_art_diffusion_soft_256","pixel_art_diffusion_hard_256","256x256_openai_comics_faces_by_alex_spirin_084000","ukiyoe_diffusion_256_022000","pixelartdiffusion_expanded","pixelartdiffusion4k","PADexpanded","watercolordiffusion","watercolordiffusion_2","PulpSciFiDiffusion","liminal_diffusion","lsun_uncond_100M_1200K_bs128"]
args.diffusion_sampling_mode = "ddim" #@param ["ddim", "plms"]
args.randomize_class = True #@param {type:"boolean"}
#@markdown ## ✂️ CLIP settings
args.cutn_batches = 4 #@param {type:"number"}
args.clip_guidance_scale =  5000#@param {type:"number"}
args.cut_overview = "[12]*400+[4]*600" #@param 
args.cut_innercut = "[4]*400+[12]*600" #@param 
args.cut_icgray_p = "[0.2]*400+[0]*600" #@param 
args.cut_ic_pow = 1 #@param {type:"number"}
args.eta = 0.8 #@param {type:"number"}
args.clamp_grad = True #@param {type:"boolean"}
args.clamp_max = 0.05 #@param {type:"number"}
args.clip_denoised = False #@param {type:"boolean"}
args.rand_mag = 0.05 #@param {type:"number"}
args.tv_scale = 0 #@param {type:"number"}
args.range_scale = 150 #@param {type:"number"}
args.sat_scale = 0 #@param {type:"number"}
args.skip_augs = False #@param {type:"boolean"}
# 🎞️ Animation Settings
args.animation_mode = "None" 
args.video_init_path = "training.mp4" 
args.extract_nth_frame = 10 
args.video_init_seed_continuity = True 
args.key_frames = True 
args.max_frames = 10000 
args.interp_spline = "Linear"
args.frames_skip_steps = "60%" 
args.vr_mode = False 
args.angle = '0:(0)' 
args.zoom = '0:(1),10:(1.05)'
args.translation_x = "0: (0)"
args.translation_y = "0: (0)" 
args.translation_z = "0: (10.0)"
args.rotation_3d_x = "0: (0)" 
args.rotation_3d_y = "0: (0)"
args.rotation_3d_z = "0: (0)" 
args.midas_depth_model = "dpt_large" 
args.midas_weight = 0.3 
args.near_plane = 0
args.far_plane = 0 
args.fov = 0 
args.padding_mode = "border" 
args.sampling_mode = "bicubic" 
args.turbo_mode = False 
args.turbo_steps = 3 
args.turbo_preroll = 10
args.frames_scale = 1500 
args.frames_skip_steps = "60%" 
#@markdown ### 🔽 Symmetry Settings
args.use_horizontal_symmetry = False #@param {type:"boolean"}
args.use_vertical_symmetry = False #@param {type:"boolean"}
args.transformation_percent = [0.09] #@param type="raw"
#@markdown ### ⚙️ Batching and DB Settings
args.modifiers = { } #@param type="raw"
args.multipliers = { } #@param type="raw"
args.save_metadata = False #@param {type:"boolean"}
args.db = "/content/gdrive/MyDrive/disco-diffusion-1/disco.db" #@param {"type":"string"}
#@markdown ### ⚙️ Appendix
#@markdown Lesser used options
#@markdown
#@markdown **`init_images`** specifies the base `init_images` folder to look for any (optional) init images.
args.init_images = "init_images" #@param {type:"string"}
#@markdown **`init_images`** specifies the base `images_out` folder to look for your batch output
args.images_out = "images_out" #@param {type:"string"}
#@markdown **`model_path`** specifies the folder where your models are downloaded to and used
args.model_path = "models" #@param {type:"string"}
#prompt_salad_path Specifies base prompt_salad paths.
args.prompt_salad_path = "prompt_salad"
args.cuda_device = "cuda:0"
#@markdown **`gen_config`** specifies the name of the generated YAML configuration file that will be saved in `configs` folder.  `AUTO` means the file will be `[batch_name]_gen.yaml` will be used.
args.gen_config = "AUTO" #@param {type:"string"}
#@markdown **`gen_config_only`** indicates the configuration only will be generated, and no batch will run.
args.gen_config_only = False #@param {type:"boolean"}
args.use_checkpoint = True #@param {type:"boolean"}
args.cutout_debug = False #@param {type:"boolean"}
args.google_drive = False
#@markdown image_prompts are supported but are not good.
args.image_prompts = { } #@param type="raw"

## Generate Image

In [ ]:
# Load defaults
dd_args.arg_configuration_loader()  # Workaround to do param saving to yaml for now.
pargs = args  # Dont run through arg_configuration_loader as it expects CLI.

# Setup folders
folders = dd.setupFolders(is_colab=dd.detectColab(), PROJECT_DIR=PROJECT_DIR, pargs=pargs)

# Load Models
dd.loadModels(folders)


# Report System Details
dd.systemDetails(pargs)

# Get CUDA Device
device = dd.getDevice(pargs)

dd.start_run(pargs=pargs, folders=folders, device=device, is_colab=dd.detectColab())


### Choose detection Folder

In [ ]:
import os
import shutil
path = folders.batch_folder
CustomImageInput = False #@param {type:"boolean"}
if CustomImageInput :
  os.chdir(path + "/../../..")
  try:
    os.mkdir("DiscoImage")
  except:
    print("Custom folder existed")
  path = os.getcwd() + "/DiscoImage" #@param {type:"raw"}

print("Your Custom Image Dir is: ")
print(path)

## Yolov5 Object Detection

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)
model.cuda()

In [ ]:
import glob

imgs = []
for img in glob.glob(path + "/*.png"):
  imgs.append(img)

result = model(imgs)
result.print()
result.show()
result.pandas().xyxy[0]

## Faster RCNN Detection

In [ ]:
import os
import numpy as np
import torch
import torchvision
import cv2
from PIL import Image
from matplotlib import pyplot as plt
from torchvision import transforms as T

model_pretrained = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Inference
model_pretrained.eval()

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
def get_prediction(img_path, threshold):
  img = Image.open(img_path) # Load the image
  transform = T.Compose([T.ToTensor()]) # Defing PyTorch Transform
  img = transform(img) # Apply the transform to the image
  pred = model_pretrained([img]) # Pass the image to the model
  if(len(pred[0]["boxes"]) == 0):
    return [],[],[]
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())] # Get the Prediction Score
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())] # Bounding boxes
  pred_score = list(pred[0]['scores'].detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  pred_score = pred_score[:pred_t+1]
  
  return pred_boxes, pred_class,pred_score


def object_detection_api(img_path, threshold=0.5, rect_th=3, text_size=1, text_th=2):
 
  boxes, pred_cls,pred_score = get_prediction(img_path, threshold) # Get predictions
  img = cv2.imread(img_path) # Read image with cv2
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
  for i in range(len(boxes)):
    cv2.rectangle(img, (int(boxes[i][0][0]),int(boxes[i][0][1])), (int(boxes[i][1][0]),int(boxes[i][1][1])),color=(0, 255, 0), thickness=rect_th) # Draw Rectangle with the coordinates
    cv2.putText(img,pred_cls[i] + str(round(pred_score[i],2)), (int(boxes[i][0][0]),int(boxes[i][0][1])),  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class


  return img,pred_cls,pred_score

In [ ]:
FasterRcnnImg = []
FRcnnCls = []
FRcnnScore = []
for img in imgs:
  out1,out2,out3 = object_detection_api(img, threshold=0.4)
  FasterRcnnImg.append(out1)
  FRcnnCls.append(out2)
  FRcnnScore.append(out3)

## Compare Objection Detection Result

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import image
import cv2
%matplotlib inline    


for i in range(len(imgs)):

  img1 = cv2.imread(imgs[i])
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
  img2 = result.imgs[i]
  img3 = FasterRcnnImg[i]
  f, grid = plt.subplots(nrows=1, ncols=3, figsize=(20, 10), dpi=120, sharex=True, sharey=True)
  grid[0].imshow(img1)
  grid[1].imshow(img2)
  grid[2].imshow(img3)

  grid[0].set_title("original")
  grid[1].set_title("Yolov5")
  grid[2].set_title("Faster R-CNN")
  plt.xticks([])
  plt.yticks([])
  plt.show()



## Generate Detection Result

In [ ]:
import pandas as pd
import numpy as np
table = dict()


#calculate result from yolov5
for eachImg in result.pandas().xyxy:
  for i in eachImg.index:
    if eachImg["name"][i] not in table:
      table[eachImg["name"][i]] = []
    table[eachImg["name"][i]].append(eachImg["confidence"][i])

#calculate result from FasterRcnn
for i in range(len(FRcnnCls)):
  for j in range(len(FRcnnCls[i])):
    className = FRcnnCls[i][j]
    classScore = FRcnnScore[i][j]
    if className not in table:
      table[className] = []
    table[className].append(classScore)

#calculate average
for eachItem in table:
  table[eachItem] = np.average(table[eachItem])

table = {k: v for k, v in sorted(table.items(),reverse = True, key=lambda item: item[1])}
print(table)

# T5 Sentence Generation

In [52]:
# auto generate 
storyLength = 5;
# get all emotion
allEmotion = [genEmo(demoStory.iloc[currentSentenceindex]['sentence']) for currentSentenceindex in range(1, storyLength)]
# get all entity
allEntity = [genKeyword(demoStory.iloc[currentSentenceindex]['sentence']) for currentSentenceindex in range(1, storyLength)]

In [53]:
from pandas import DataFrame
DataFrame(allEmotion).to_csv("./promptChange/allEmotion.csv")
DataFrame(allEntity).to_csv("./promptChange/allEntity.csv")

In [64]:
# Input Construction
def getInput(storyContext, currentSentenceindex):
    return f"Generate next sentence based on:<extra_id_0>KEYWORDS: {allEntity[currentSentenceindex-1]}" + f"<extra_id_1>CONTEXT: {' '.join(storyContext)}" + f"<extra_id_1>EMOTION: {allEmotion[currentSentenceindex-1]}"

def getInputNoPrompt(storyContext, currentSentenceindex):
    return f"{allEntity[currentSentenceindex-1]} <extra_id_0>{' '.join(storyContext)}<extra_id_1>{allEmotion[currentSentenceindex-1]}"

# Input Construction
def getInputOptimized(storyContext, currentSentenceindex):
    return f"Generate next sentence that makes reader feels {allEmotion[currentSentenceindex-1]}.<extra_id_0>KEYWORD: {allEntity[currentSentenceindex-1]}<extra_id_1>CONTEXT: {' '.join(storyContext)}"

# Input Construction
def getInputContextForward(storyContext, currentSentenceindex):
    return f"{' '.join(storyContext)}<extra_id_0>Generate next sentence based on: KEYWORD:{allEntity[currentSentenceindex-1]}<extra_id_1>EMOTION {allEmotion[currentSentenceindex-1]}"

In [78]:
import transformers
import evaluate
from transformers import T5Tokenizer, T5ForConditionalGeneration
# ("mrm8488/t5-base-finetuned-common_gen")
# tokenizerPrompted = T5Tokenizer.from_pretrained("Yuetian/T5-finetuned-storyCommonsense")
# modelPrompted = T5ForConditionalGeneration.from_pretrained("Yuetian/T5-finetuned-storyCommonsense")
# tokenizerNoPrompted = T5Tokenizer.from_pretrained("Yuetian/T5-finetuned-storyCommonsense-noPrompt")
# modelNoPrompted = T5ForConditionalGeneration.from_pretrained("Yuetian/T5-finetuned-storyCommonsense-noPrompt")
# tokenizerOptimizedPrompted = T5Tokenizer.from_pretrained("Yuetian/T5-finetuned-storyCommonsense-noPrompt")
# modelOptimizedPrompted = T5ForConditionalGeneration.from_pretrained("Yuetian/T5-finetuned-storyCommonsense-noPrompt")

tokenizerPrompted = T5Tokenizer.from_pretrained("D:/models/promptComparison/defaultPrompt")
modelPrompted = T5ForConditionalGeneration.from_pretrained("D:/models/promptComparison/defaultPrompt")
tokenizerNoPrompted = T5Tokenizer.from_pretrained("D:/models/promptComparison/noPrompt")
modelNoPrompted = T5ForConditionalGeneration.from_pretrained("D:/models/promptComparison/noPrompt")
tokenizerOptimizedPrompted = T5Tokenizer.from_pretrained("D:/models/promptComparison/optimizedPrompt")
modelOptimizedPrompted = T5ForConditionalGeneration.from_pretrained("D:/models/promptComparison/optimizedPrompt")
tokenizerContextForward = T5Tokenizer.from_pretrained("D:/models/promptComparison/contextForward")
modelContextForward = T5ForConditionalGeneration.from_pretrained("D:/models/promptComparison/contextForward")

model = AutoModelForSequenceClassification.from_pretrained("Yuetian/roberta-large-finetuned-plutchik-emotion")
tokenizer = AutoTokenizer.from_pretrained("Yuetian/roberta-large-finetuned-plutchik-emotion")

In [79]:
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")
sacrebleu = evaluate.load("sacrebleu")

In [80]:
def genSentenceWithPrompt(words, max_length=32):
    input_text = words
    features = tokenizerPrompted([input_text], return_tensors='pt')
    # print("/tinput: " + input_text)
    output = modelPrompted.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            max_length=max_length)
    return tokenizerPrompted.decode(output[0], skip_special_tokens=True)

def genSentenceWithoutPrompt(words, max_length=32):
    input_text = words
    features = tokenizerNoPrompted([input_text], return_tensors='pt')
    # print("/tinput: " + input_text)
    output = modelNoPrompted.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            max_length=max_length)
    return tokenizerNoPrompted.decode(output[0], skip_special_tokens=True)

def genSentenceWithOptimizedPrompt(words, max_length=32):
    input_text = words
    features = tokenizerPrompted([input_text], return_tensors='pt')
    # print("/tinput: " + input_text)
    output = modelOptimizedPrompted.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            max_length=max_length)
    return tokenizerOptimizedPrompted.decode(output[0], skip_special_tokens=True)

def genSentenceWithContextForward(words, max_length=32):
    input_text = words
    features = tokenizerPrompted([input_text], return_tensors='pt')
    # print("/tinput: " + input_text)
    output = modelContextForward.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            max_length=max_length)
    return tokenizerContextForward.decode(output[0], skip_special_tokens=True)

def genStoryNewModel(story_length = storyLength):
    story = []
    story.append(demoSentence);
    # print("PROMPT INPUT:")
    for i in range(1, storyLength):
        story.append(genSentenceWithPrompt(getInput(story, i)))
    return story

def genStoryOptmizedNewModel(story_length = storyLength):
    story = []
    story.append(demoSentence);
    # print("PROMPT INPUT:")
    for i in range(1, storyLength):
        story.append(genSentenceWithOptimizedPrompt(getInputOptimized(story, i)))
    return story

def genStoryContextForward(story_length = storyLength):
    story = []
    story.append(demoSentence);
    # print("PROMPT INPUT:")
    for i in range(1, storyLength):
        story.append(genSentenceWithContextForward(getInputContextForward(story, i)))
    return story

def genStoryOriginalModel(story_length = storyLength):
    story = []
    story.append(demoSentence);
    # print("NO PROMPT INPUT:")
    for i in range(1, storyLength):
        story.append(genSentenceWithoutPrompt(getInputNoPrompt(story, i)))
    return story

def BLEU_metric(generatedText, actualText):
    results = bleu.compute(predictions= generatedText, references= actualText)
    return results

def BERT_metric(generatedText, actualText):
    results = bertscore.compute(predictions=generatedText, references=actualText, lang="en") #, model_type="distilbert-base-uncased"
    return results

def sacrebleu_metric(generatedText, actualText):
    results = sacrebleu.compute(predictions= generatedText, references= actualText)
    return results

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
import torch

# seq: joy, trust, fear, surprise, sadness, disgust, anger, anticipation
def emoMatchMetric(expect, actual):
    # model = AutoModelForSequenceClassification.from_pretrained("./firstSentenceDetection/") # local
    # model = AutoModelForSequenceClassification.from_pretrained("Yuetian/bert-base-uncased-finetuned-plutchik-emotion")
    # assert model.config.output_attentions == True
    # tokenizer = AutoTokenizer.from_pretrained("Yuetian/bert-base-uncased-finetuned-plutchik-emotion")
    encoding = tokenizer(expect, return_tensors="pt")
    # tokenizer.decode(encoding['input_ids'])
    encoding = {k: v.to('cpu') for k,v in encoding.items()}
    outputs = model(**encoding)   
    # apply sigmoid + threshold
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probsExpect = sigmoid(logits.squeeze().cpu())
    
    encoding = tokenizer(actual, return_tensors="pt")
    # tokenizer.decode(encoding['input_ids'])
    encoding = {k: v.to('cpu') for k,v in encoding.items()}
    outputs = model(**encoding)   
    # apply sigmoid + threshold
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probsActual = sigmoid(logits.squeeze().cpu())
    
    probsExpect = probsExpect.detach().numpy()
    probsActual = probsActual.detach().numpy()
    # return [probsExpect, probsActual]
    return (np.linalg.norm(np.array([1, 1, 1, 1, 1, 1, 1, 1]) - np.array([0, 0, 0, 0, 0, 0, 0, 0])) - np.linalg.norm(probsExpect - probsActual))/np.linalg.norm(np.array([1, 1, 1, 1, 1, 1, 1, 1]) - np.array([0, 0, 0, 0, 0, 0, 0, 0])) 

In [81]:
def getBLEUEachSentence(actual, expected):
    return [BLEU_metric([x], [y])['bleu'] for x, y in zip(list(actual), list(expected))]

def getMeteorEachSentence(actual, expected):
    return [meteor([word_tokenize(x)], word_tokenize(y)) for x, y in zip(list(actual), list(expected))]

def getBertEachSentence(actual, expected):   
    return [BERT_metric([x], [y])['precision'][0] for x, y in zip(list(actual), list(expected))]

def getSacrebleuEachSentence(actual, expected):   
    return [sacrebleu_metric([x], [y])['score'] for x, y in zip(list(actual), list(expected))]

def getEmotionScoreEachSentence(actual, expected):
    return [emoMatchMetric([x], [y]) for x, y in zip(list(actual), list(expected))]

In [82]:
from pandas import DataFrame
from nltk.translate import meteor
from nltk import word_tokenize
# generate
allEmotion = list(pd.read_csv("./promptChange/allEmotion.csv")['0'])
allEntity = list(pd.read_csv("./promptChange/allEntity.csv")['0'])

generartedReport = DataFrame({"generated - prompt" : genStoryNewModel(), 
                "generated - no prompt" : genStoryOriginalModel(), "ground truth": demoStory['sentence'].tolist()})
generartedReport['BLEU - no prompt'] = getBLEUEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
generartedReport['BLEU - prompt'] = getBLEUEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
generartedReport['METEOR - no prompt'] = getMeteorEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
generartedReport['METEOR - prompt'] = getMeteorEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
generartedReport['BERT - no prompt'] = getBertEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
generartedReport['BERT - prompt'] = getBertEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
generartedReport['SacreBLEU - no prompt'] = getSacrebleuEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
generartedReport['SacreBLEU - prompt'] = getSacrebleuEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
# generartedReport.to_csv(f"./drive/MyDrive/results/baselineCompare/result.csv") # remote

In [20]:
generartedReport.to_csv("./changed.csv")
generartedReport.style.set_properties(subset=[], **{'width-min': '300px'})

,generated - prompt,generated - no prompt,ground truth,BLEU - no prompt,BLEU - prompt,METEOR - no prompt,METEOR - prompt,BERT - no prompt,BERT - prompt,SacreBLEU - no prompt,SacreBLEU - prompt
0,I wanted to do a different thing than the usual.,I wanted to do a different thing than the usual.,I wanted to do a different thing than the usual.,1.000000,1.000000,0.999624,0.999624,1.000000,1.000000,100.000000,100.000000
1,I got into my car and went on the road.,I decided to do a dance.,I got into my car and went on the road.,0.000000,1.000000,0.135135,0.999624,0.933325,1.000000,4.410364,100.000000
2,I stopped at a blockbuster video store.,I did a lot of choreography.,I stopped at a blockbuster video store.,0.000000,1.000000,0.211268,0.999023,0.890072,1.000000,7.492443,100.000000
3,I went inside and rented a movie.,I did a lot of choreography.,I went inside and rented a movie.,0.000000,1.000000,0.211268,0.999023,0.899168,1.000000,7.492443,100.000000
4,I brought the movie home and watched the whole thing.,I did a dance.,I brought the movie home and watched the whole thing.,0.000000,1.000000,0.178571,0.999624,0.934899,1.000000,3.826166,100.000000


# Experiments
## Split all 5 sentence story
Baseline vs. fine-tuned

In [83]:
# local
import pandas as pd
# dfTrain = pd.read_csv('./A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability/data/csv_version/dev/emotion/allcharlinepairs-clean-v1.csv')
# dfTest  = pd.read_csv('./A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability/data/csv_version/test/emotion/allcharlinepairs-clean-v1.csv')
dfTrain = pd.read_csv('../../data/csv_version/dev/emotion/allcharlinepairs-clean-v1.csv') # local
dfTest  = pd.read_csv('../../data/csv_version/test/emotion/allcharlinepairs-clean-v1.csv') # local
dfAll = pd.concat([dfTest, dfTrain]).reset_index(drop=True) # shuffle and combine everything, ratio issue fix here!
dfAll

,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
0,0,a3e66276-ef27-426f-8a2f-329779f17038,1,Cook,ann0,NaN,A cook was carrying an armful of oranged in th...,yes,"['joy', 'useful', 'nervous']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ..."
1,1,a3e66276-ef27-426f-8a2f-329779f17038,2,Cook,ann0,A cook was carrying an armful of oranged in th...,He dropped one on the floor by accident.,yes,"['annoyed', 'angry', 'frustrated']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ..."
2,2,a3e66276-ef27-426f-8a2f-329779f17038,3,Cook,ann0,A cook was carrying an armful of oranged in th...,As a joke he pretended that it was a soccer ball.,yes,"['amused', 'playful', 'embarrassed']","{'joy': 1, 'trust': 1, 'fear': 0, 'surprise': ..."
3,3,a3e66276-ef27-426f-8a2f-329779f17038,4,Cook,ann0,A cook was carrying an armful of oranged in th...,He kicked the orange across the kitchen.,yes,"['playful', 'angry']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ..."
4,4,a3e66276-ef27-426f-8a2f-329779f17038,5,Cook,ann0,A cook was carrying an armful of oranged in th...,It landed in a pot and he cheered!,yes,"['great', 'amused', 'surprised', 'delighted']","{'joy': 3, 'trust': 0, 'fear': 0, 'surprise': ..."
...,...,...,...,...,...,...,...,...,...,...
22734,11605,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,1,Marcus,ann0,NaN,Marcus was collecting shells on the beach.,yes,"['intrigued', 'happy', 'contemplative']","{'joy': 3, 'trust': 1, 'fear': 0, 'surprise': ..."
22735,11606,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,2,Marcus,ann0,Marcus was collecting shells on the beach.,He picked up a large beautiful shell.,yes,"['awestruck', 'moved', 'joy']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ..."
22736,11607,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,3,Marcus,ann0,Marcus was collecting shells on the beach.|He ...,He put it in his pocket to save for later.,yes,"['satisfied', 'excited', 'curiosity']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ..."
22737,11608,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,4,Marcus,ann0,Marcus was collecting shells on the beach.|He ...,Suddenly he felt a sharp pinch.,yes,"['pained', 'sore', 'surprised']","{'joy': 1, 'trust': 0, 'fear': 2, 'surprise': ..."


In [84]:
# simplify storyid for easy iteratrion
previousStoryId = dfAll.iloc[0]['storyid']
iterStoryId = 0
iterStoryEachLine = 0
iterStoryInvalidCount = 0;
for index, row in dfAll.iterrows():
    # print("before:", iterStoryEachLine)
    if (previousStoryId == dfAll.iloc[index]['storyid']):
        dfAll.loc[index, 'shortstoryid'] = int(iterStoryId);
        iterStoryEachLine += 1
        # print(iterStoryEachLine)
    else:
        if iterStoryEachLine != 5: # case: find invalid story   
            dfAll.loc[index-iterStoryEachLine : index-1, 'shortstoryid'] = -1;
            iterStoryInvalidCount+=1 
        else:
            iterStoryId += 1  
        previousStoryId = dfAll.iloc[index]['storyid']
        dfAll.loc[index, 'shortstoryid'] = int(iterStoryId)
        iterStoryEachLine = 1
        
dfAll['shortstoryid'] = dfAll['shortstoryid'].astype(int)
print(f"Stat: \n\t invalid: {iterStoryInvalidCount}\n\t valid: {iterStoryId + 1}")

Stat: 
	 invalid: 1080
	 valid: 3749


In [85]:
dfAll[:20]

,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik,shortstoryid
0,0,a3e66276-ef27-426f-8a2f-329779f17038,1,Cook,ann0,NaN,A cook was carrying an armful of oranged in th...,yes,"['joy', 'useful', 'nervous']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ...",0
1,1,a3e66276-ef27-426f-8a2f-329779f17038,2,Cook,ann0,A cook was carrying an armful of oranged in th...,He dropped one on the floor by accident.,yes,"['annoyed', 'angry', 'frustrated']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",0
2,2,a3e66276-ef27-426f-8a2f-329779f17038,3,Cook,ann0,A cook was carrying an armful of oranged in th...,As a joke he pretended that it was a soccer ball.,yes,"['amused', 'playful', 'embarrassed']","{'joy': 1, 'trust': 1, 'fear': 0, 'surprise': ...",0
3,3,a3e66276-ef27-426f-8a2f-329779f17038,4,Cook,ann0,A cook was carrying an armful of oranged in th...,He kicked the orange across the kitchen.,yes,"['playful', 'angry']","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ...",0
4,4,a3e66276-ef27-426f-8a2f-329779f17038,5,Cook,ann0,A cook was carrying an armful of oranged in th...,It landed in a pot and he cheered!,yes,"['great', 'amused', 'surprised', 'delighted']","{'joy': 3, 'trust': 0, 'fear': 0, 'surprise': ...",0
5,5,5a49eb64-7560-4d40-a9c6-9f5ea7dce513,1,Valerie,ann0,NaN,Valerie was getting ready for a formal dance.,yes,"['thrilled', 'excited']","{'joy': 2, 'trust': 1, 'fear': 2, 'surprise': ...",1
6,6,5a49eb64-7560-4d40-a9c6-9f5ea7dce513,2,Valerie,ann0,Valerie was getting ready for a formal dance.,She had been preparing for hours.,yes,"['anxious', 'ready', 'excited']","{'joy': 2, 'trust': 1, 'fear': 1, 'surprise': ...",1
7,7,5a49eb64-7560-4d40-a9c6-9f5ea7dce513,3,Valerie,ann0,Valerie was getting ready for a formal dance.|...,"As she was ready to leave, her acrylic nail br...",yes,"['upset', 'sad', 'devastated']","{'joy': 0, 'trust': 0, 'fear': 1, 'surprise': ...",1
8,8,5a49eb64-7560-4d40-a9c6-9f5ea7dce513,4,Valerie,ann0,Valerie was getting ready for a formal dance.|...,She snapped off all of her faux nails.,yes,"['silly', 'pain', 'angry']","{'joy': 0, 'trust': 1, 'fear': 1, 'surprise': ...",1
9,9,5a49eb64-7560-4d40-a9c6-9f5ea7dce513,5,Valerie,ann0,Valerie was getting ready for a formal dance.|...,She didn't have time to feel pity about them.,yes,"['uncomfortable', 'frustrated', 'angry']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",1


## Start fetch data

In [86]:
import torch
use_cuda = torch.cuda.is_available()

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8302
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 3080
__CUDA Device Total Memory [GB]: 10.736893952


In [89]:
from pandas import DataFrame
from nltk.translate import meteor
from nltk import word_tokenize
for index in range(200,3749):
    #try:    
      # for index in range(1):
      # find story
      demoStory = dfAll[dfAll['shortstoryid'] == index]
      demoSentence = demoStory.iloc[0]['sentence']
      # print(demoStory)
      print("\rcurrent progress: {}/{} ".format(index+1, iterStoryId+1), end="")
      # update prompt
      allEmotion = [genEmo(demoStory.iloc[currentSentenceindex]['sentence']) for currentSentenceindex in range(1, storyLength)]
      allEntity = [genKeyword(demoStory.iloc[currentSentenceindex]['sentence']) for currentSentenceindex in range(1, storyLength)]
      
      # generate
      generartedReport = DataFrame({"generated - prompt" : genStoryNewModel(), 
                                  "generated - no prompt" : genStoryOriginalModel(), "generated - optimized prompt" : genStoryOptmizedNewModel(),
                                  "generated - context forward" : genStoryContextForward(),
                                  "ground truth": demoStory['sentence'].tolist()})
      generartedReport['BLEU - no prompt'] = getBLEUEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
      generartedReport['BLEU - prompt'] = getBLEUEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
      generartedReport['BLEU - optimized prompt'] = getBLEUEachSentence(generartedReport["generated - optimized prompt"], generartedReport["ground truth"])
      generartedReport['BLEU - context forward'] = getBLEUEachSentence(generartedReport["generated - optimized prompt"], generartedReport["ground truth"])
    
      generartedReport['METEOR - no prompt'] = getMeteorEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
      generartedReport['METEOR - prompt'] = getMeteorEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
      generartedReport['METEOR - optimized prompt'] = getMeteorEachSentence(generartedReport["generated - optimized prompt"], generartedReport["ground truth"])
      generartedReport['METEOR - context forward'] = getMeteorEachSentence(generartedReport["generated - context forward"], generartedReport["ground truth"])
    
      generartedReport['BERT - no prompt'] = getBertEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
      generartedReport['BERT - prompt'] = getBertEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
      generartedReport['BERT - optimized prompt'] = getBertEachSentence(generartedReport["generated - optimized prompt"], generartedReport["ground truth"])
      generartedReport['BERT - context forward'] = getBertEachSentence(generartedReport["generated - context forward"], generartedReport["ground truth"]) 
    
      generartedReport['SacreBLEU - no prompt'] = getSacrebleuEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
      generartedReport['SacreBLEU - prompt'] = getSacrebleuEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
      generartedReport['SacreBLEU - optimized prompt'] = getSacrebleuEachSentence(generartedReport["generated - optimized prompt"], generartedReport["ground truth"])
      generartedReport['SacreBLEU - context forward'] = getSacrebleuEachSentence(generartedReport["generated - context forward"], generartedReport["ground truth"])
    
      generartedReport['Emotion - prompt'] = getEmotionScoreEachSentence(generartedReport["generated - prompt"], generartedReport["ground truth"])
      generartedReport['Emotion - no prompt'] = getEmotionScoreEachSentence(generartedReport["generated - no prompt"], generartedReport["ground truth"])
      generartedReport['Emotion - optimized prompt'] = getEmotionScoreEachSentence(generartedReport["generated - optimized prompt"], generartedReport["ground truth"])
      generartedReport['Emotion - context forward'] = getEmotionScoreEachSentence(generartedReport["generated - context forward"], generartedReport["ground truth"])
      # generartedReport.to_csv(f"./drive/MyDrive/results/baselineCompare/result-{index}.csv") # remote
      generartedReport.to_csv(f"./baselineCompare/result-{index}.csv") #local
    #except:
      #continue
    # generartedReport.to_csv(f"./results/baselineCompare/result-{index}.csv") #local
    # generartedReport.style.set_properties(subset=[], **{'width-min': '300px'})
    # generartedReport


current progress: 3749/3749 

In [ ]:
genrartedReport

,generated - prompt,generated - no prompt,ground truth,BLEU - no prompt,BLEU - prompt,METEOR - no prompt,METEOR - prompt,BERT - no prompt,BERT - prompt,SacreBLEU - no prompt,SacreBLEU - prompt
0,A cook was carrying an armful of oranged in th...,A cook was carrying an armful of oranged in th...,A cook was carrying an armful of oranged in th...,1.0,1.000000,0.999711,0.999711,[1.0000001192092896],[1.0000001192092896],100.000000,100.000000
1,"Suddenly, he dropped one on the floor by accid...",A man was trying to cut the orange from his hand.,He dropped one on the floor by accident.,0.0,0.678650,0.085470,0.832762,[0.8803496956825256],[0.9710865020751953],4.027248,67.865027
2,During a joke he pretended that it was a socce...,The man slipped and fell.,As a joke he pretended that it was a soccer ball.,0.0,0.903602,0.075758,0.916322,[0.91267329454422],[0.9832724332809448],2.985966,90.360200
3,"Suddenly, he kicked the orange across the kitc...",The man was rushed to the hospital.,He kicked the orange across the kitchen.,0.0,0.638943,0.250000,0.815529,[0.9011991024017334],[0.9746093153953552],6.567275,63.894310
4,landed in a pot and he cheered!,The man was able to get the orange out of his ...,It landed in a pot and he cheered!,0.0,0.882497,0.000000,0.986690,[0.8726325035095215],[0.9620723128318787],0.000000,88.249690
